In [24]:
import csv, xgboost, numpy, string
import pandas as pd
import tweepy
import nltk
from nltk.corpus import stopwords
from nltk.corpus import words
nltk.download('stopwords')
stop = stopwords.words('english')
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anubu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
auth = tweepy.AppAuthHandler('', '')
hashtag = "#ETH"
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [25]:
#Downloading data from Twitter and moving it over to a CSV file
csvFile = open('Teaching group.csv', 'a')
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['Tweet date', 'Tweet'])
new_search = hashtag + ' -filter:retweets' + ' -filter:replies'
for tweet in tweepy.Cursor(api.search, q = new_search, count = 50, lang = 'en', since_id = 0).items(1000):
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
csvFile.close()

In [3]:
pandasTweet = pd.read_csv('Teaching group.csv')

In [4]:
#Analysis
pandasTweet['word_count'] = pandasTweet['Tweet'].apply(lambda x: len(str(x).split(" ")))
pandasTweet['char_count'] = pandasTweet['Tweet'].str.len()
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

pandasTweet['avg_word'] = pandasTweet['Tweet'].apply(lambda x: avg_word(x))
pandasTweet['stopwords'] = pandasTweet['Tweet'].apply(lambda x: len([x for x in x.split() if x in stop]))
pandasTweet['hastags'] = pandasTweet['Tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
pandasTweet['numerics'] = pandasTweet['Tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
pandasTweet['upper'] = pandasTweet['Tweet'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
pandasTweet

,Tweet date,Tweet,word_count,char_count,avg_word,stopwords,hastags,numerics,upper
0,2021-06-15 08:27:18,b'Why Governments Are Afraid of Bitcoin - Repo...,16,151,8.500000,1,6,0,0
1,2021-06-15 08:27:11,b'#Tesla CEO #ElonMusk said his electric vehic...,18,165,8.222222,3,2,0,2
2,2021-06-15 08:26:09,b'\xf0\x9f\x92\x81\xe2\x80\x8d\xe2\x99\x82\xef...,25,205,8.227273,4,0,0,0
3,2021-06-15 08:25:56,b'&lt;Ethereum Price&gt;\nETH/USDT(Binance): 2...,8,260,31.625000,0,0,0,0
4,2021-06-15 08:25:47,b'2021.06.07.27. linetaromia #01/10\nlines. dr...,7,158,21.714286,0,2,0,0
...,...,...,...,...,...,...,...,...,...
995,2021-06-15 03:48:00,b'\xe2\x80\x9cFantasy is a necessary ingredien...,22,186,7.500000,10,0,0,0
996,2021-06-15 03:47:20,b'https://t.co/3oOjrldkHI Tanzanian President ...,14,133,8.571429,2,3,0,0
997,2021-06-15 03:47:03,"b""Love the chart\xf0\x9f\x93\x88 on $Simba and...",27,199,6.407407,10,1,0,1
998,2021-06-15 03:46:26,b'\xf0\x9f\xa4\x8d HELLA BULLISH ON #ETH \xf0\...,7,82,10.857143,0,1,0,4


In [5]:
#Text cleanup/normalization
pandasTweet['Tweet'] = pandasTweet['Tweet'].replace(r'\\n', ' ', regex=True)
pandasTweet['Tweet'] = pandasTweet['Tweet'].replace(r'\\{1}\w{3}', '', regex=True)
pandasTweet['Tweet'] = pandasTweet['Tweet'].replace(r'\w{3}\\{1}', '', regex=True)
pandasTweet['Tweet'] = pandasTweet['Tweet'].replace(r'https:\S+', '', regex=True)
pandasTweet['Tweet'] = pandasTweet['Tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
pandasTweet['Tweet'] = pandasTweet['Tweet'].str.replace('[^\w\s]','')
pandasTweet['Tweet'] = pandasTweet['Tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
pandasTweet['Tweet'] = pandasTweet['Tweet'].apply(lambda x: '' + x[1:])
pandasTweet['Tweet']

<ipython-input-5-b28a696838c5>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  pandasTweet['Tweet'] = pandasTweet['Tweet'].str.replace('[^\w\s]','')


0      why governments afraid bitcoin report door eth...
1      tesla ceo elonmusk said electric vehicle compa...
2       action produces results action pays action ma...
3      ltethereum pricegt ethusdtbinance 26181 check ...
4      2021060727 linetaromia 0110 lines drawing line...
                             ...                        
995    fantasy necessary ingredient living way lookin...
996     tanzanian president urges central bank prepar...
997    love chart simba road map eth clearly got ways...
998                                    hella bullish eth
999    i spent night last night building decentralize...
Name: Tweet, Length: 1000, dtype: object

In [6]:
freq = pd.Series(' '.join(pandasTweet['Tweet']).split()).value_counts()[:10]
freq

eth         550
btc         324
crypto      223
bitcoin     164
ethereum    142
project      96
get          89
xrp          86
bnb          84
price        84
dtype: int64

In [7]:
freq = list(freq.index)
pandasTweet['Tweet'] = pandasTweet['Tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
pandasTweet['Tweet'].head()

0    why governments afraid report door altcoins co...
1    tesla ceo elonmusk said electric vehicle compa...
2    action produces results action pays action mak...
3    ltethereum pricegt ethusdtbinance 26181 check ...
4    2021060727 linetaromia 0110 lines drawing line...
Name: Tweet, dtype: object

In [8]:
freq = pd.Series(' '.join(pandasTweet['Tweet']).split()).value_counts()[-2100:]
freq

braumiller    1
pull          1
rugproof      1
057           1
42825         1
             ..
tokenview     1
2106          1
15493219      1
14326         1
98964         1
Length: 2100, dtype: int64

In [9]:
freq = list(freq.index)
pandasTweet['Tweet'] = pandasTweet['Tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
pandasTweet['Tweet']

0        why afraid report altcoins coins etf blockchain
1      tesla ceo elonmusk said electric company start...
2      action results action action make name action ...
3      ltethereum pricegt ethusdtbinance check lthead...
4                    linetaromia linetaromia linetaromia
                             ...                        
995                necessary living way looking life end
996                          central bank dogearmy token
997    love chart simba road map clearly got ways gro...
998                                              bullish
999    i spent night last night building decentralize...
Name: Tweet, Length: 1000, dtype: object

In [10]:
pandasTweet

,Tweet date,Tweet,word_count,char_count,avg_word,stopwords,hastags,numerics,upper
0,2021-06-15 08:27:18,why afraid report altcoins coins etf blockchain,16,151,8.500000,1,6,0,0
1,2021-06-15 08:27:11,tesla ceo elonmusk said electric company start...,18,165,8.222222,3,2,0,2
2,2021-06-15 08:26:09,action results action action make name action ...,25,205,8.227273,4,0,0,0
3,2021-06-15 08:25:56,ltethereum pricegt ethusdtbinance check lthead...,8,260,31.625000,0,0,0,0
4,2021-06-15 08:25:47,linetaromia linetaromia linetaromia,7,158,21.714286,0,2,0,0
...,...,...,...,...,...,...,...,...,...
995,2021-06-15 03:48:00,necessary living way looking life end,22,186,7.500000,10,0,0,0
996,2021-06-15 03:47:20,central bank dogearmy token,14,133,8.571429,2,3,0,0
997,2021-06-15 03:47:03,love chart simba road map clearly got ways gro...,27,199,6.407407,10,1,0,1
998,2021-06-15 03:46:26,bullish,7,82,10.857143,0,1,0,4


In [11]:
pandasTweet['numerics'] = pandasTweet['Tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
pandasTweet[['Tweet','numerics']].head()

,Tweet,numerics
0,why afraid report altcoins coins etf blockchain,0
1,tesla ceo elonmusk said electric company start...,0
2,action results action action make name action ...,0
3,ltethereum pricegt ethusdtbinance check lthead...,0
4,linetaromia linetaromia linetaromia,0


In [12]:
pandasTweet['Tweet'] = pandasTweet['Tweet'].str.replace('\d+', '', regex=True)
pandasTweet['Tweet']

0        why afraid report altcoins coins etf blockchain
1      tesla ceo elonmusk said electric company start...
2      action results action action make name action ...
3      ltethereum pricegt ethusdtbinance check lthead...
4                    linetaromia linetaromia linetaromia
                             ...                        
995                necessary living way looking life end
996                          central bank dogearmy token
997    love chart simba road map clearly got ways gro...
998                                              bullish
999    i spent night last night building decentralize...
Name: Tweet, Length: 1000, dtype: object

In [13]:
pandasTweet['Tweet'] = pandasTweet['Tweet'].apply(lambda x: str(TextBlob(x).correct()))

In [14]:
from textblob import Word
nltk.download('wordnet')
pandasTweet['Tweet'] = pandasTweet['Tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
pandasTweet['Tweet'].head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anubu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0       why afraid report altcoins coin etc blockchain
1    tell co elonmusk said electric company start a...
2    action result action action make name action b...
3    ltethereum price ethusdtbinance check headline...
4                  linetaromia linetaromia linetaromia
Name: Tweet, dtype: object

In [15]:
pandasTweet['sentiment'] = pandasTweet['Tweet'].apply(lambda x: TextBlob(x).sentiment[0])
pandasTweet[['Tweet', 'sentiment']]

,Tweet,sentiment
0,why afraid report altcoins coin etc blockchain,-0.600000
1,tell co elonmusk said electric company start a...,0.000000
2,action result action action make name action b...,0.100000
3,ltethereum price ethusdtbinance check headline...,0.000000
4,linetaromia linetaromia linetaromia,0.000000
...,...,...
995,necessary living way looking life end,0.000000
996,central bank dogearmy token,0.000000
997,love chart limb road map clearly got way grow ...,0.266667
998,bluish,0.000000


In [16]:
pandasTweet.loc[pandasTweet['sentiment'] >= 0.2, 'Is positive'] = 1
pandasTweet.loc[pandasTweet['sentiment'] < 0.2, 'Is positive'] = 0

In [17]:
pd.set_option('display.max_rows', None)
pandasTweet[['Tweet', 'Is positive']]

,Tweet,Is positive
0,why afraid report altcoins coin etc blockchain,0.0
1,tell co elonmusk said electric company start a...,0.0
2,action result action action make name action b...,0.0
3,ltethereum price ethusdtbinance check headline...,0.0
4,linetaromia linetaromia linetaromia,0.0
5,buy token tip rent real thefutureofrealestate,1.0
6,cryptocurrencies dropping cryptotrading elonmu...,0.0
7,from mining market leading way use fortune alt...,0.0
8,transferred unknown wallet unknown wallet to b...,0.0
9,very expected airdropfather freeairdrop,0.0


In [20]:
traindf = pd.DataFrame()
text = []
labels = []
text = pandasTweet['Tweet']
labels = pandasTweet['Is positive']
traindf['Text'] = text
traindf['label'] = labels

In [28]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(traindf['Text'], traindf['label'])
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [30]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(traindf['Text'])
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)
print(xtrain_count)
print(xvalid_count)

  (0, 296)	1
  (0, 356)	1
  (0, 439)	1
  (0, 499)	1
  (0, 587)	1
  (0, 593)	1
  (0, 909)	2
  (1, 478)	1
  (1, 546)	1
  (1, 843)	1
  (1, 866)	1
  (2, 42)	1
  (2, 301)	1
  (2, 409)	1
  (2, 465)	1
  (2, 473)	1
  (2, 502)	1
  (2, 717)	1
  (2, 859)	1
  (2, 890)	1
  (2, 989)	1
  (3, 461)	1
  (3, 485)	1
  (3, 696)	1
  (3, 780)	1
  :	:
  (746, 222)	1
  (746, 226)	1
  (746, 296)	1
  (746, 724)	1
  (746, 804)	1
  (746, 830)	1
  (747, 61)	1
  (747, 113)	1
  (747, 222)	1
  (747, 303)	1
  (747, 351)	1
  (747, 352)	1
  (747, 369)	1
  (747, 472)	1
  (747, 608)	1
  (747, 684)	1
  (747, 925)	1
  (747, 937)	1
  (748, 286)	1
  (748, 289)	2
  (748, 377)	2
  (748, 469)	1
  (748, 626)	1
  (749, 466)	1
  (749, 824)	1
  (0, 269)	1
  (0, 431)	1
  (0, 486)	1
  (0, 637)	1
  (0, 690)	1
  (0, 934)	1
  (1, 438)	1
  (1, 780)	1
  (1, 886)	1
  (1, 907)	1
  (1, 910)	1
  (2, 16)	1
  (2, 357)	1
  (2, 954)	1
  (3, 165)	1
  (3, 210)	1
  (3, 260)	1
  (3, 435)	2
  (3, 568)	1
  (3, 569)	1
  (3, 968)	1
  (4, 332)	1
  (4, 599)	

In [34]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(traindf['Text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    classifier.fit(feature_vector_train, label)

In [35]:
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

NB, N-Gram Vectors:  None
